In [2]:
from scripts.round3.figures_import_helper_r3 import *

In [11]:
list_data_sets = [
    ('Bridges', 'bc_collusion.csv'),
    ('Electric', 'ec_collusion.csv'),
    ('Pre-Stressed Concrete', 'pc_collusion.csv'),
    ('Floods', 'fc_collusion.csv')
]

In [19]:
for name, file in list_data_sets:
    data = asymptotics.MultistagePIDMeanAuctionData(
    os.path.join(path_data, file))
    data_before = asymptotics.MultistagePIDMeanAuctionData.from_clean_bids(
    data.df_bids.loc[data.data.before == 1])
    data_after = asymptotics.MultistagePIDMeanAuctionData.from_clean_bids(
    data.df_bids.loc[data.data.before.isnull()])

    print(name, '\n\n sample size\t', data_after.df_auctions.shape[0])
    print(data_after.demand_vector(all_deviations), '\n')
    
    solver = asymptotics.AsymptoticMultistageSolver(
        data=data, 
        deviations=all_deviations, 
        metric=rebidding.EfficientMultistageIsNonCompetitive, 
        plausibility_constraints=[environments.EmptyConstraint()],
        num_points=1000, moment_matrix=multistage_moment_matrix,                 
        confidence_level=.95, enhanced_guesses=True
    )
    print(' tolerances\n', solver.tolerance, '\n')

Bridges 

 sample size	 8
win_down     0.260038
marg_cont    0.000000
marg_info    0.000000
win0         0.195644
win_up       0.184280
dtype: float64 

 tolerances
 [[-0.01282379]
 [ 0.12498438]
 [-0.0594389 ]
 [ 0.0046897 ]] 



/home/sylvain/workspace/missing_bids/mb_api/asymptotics.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bids[col] = self._get_new_wins(df_bids, rho)
/home/sylvain/workspace/missing_bids/mb_api/asymptotics.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bids['marg_cont'] = self.is_marginal_cont(df_bids, down_dev)
/home/sylvain/workspace/missing_bids/mb_api/asymptotics.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

Electric 

 sample size	 128
win_down     0.252043
marg_cont    0.011359
marg_info    0.017727
win0         0.232084
win_up       0.229666
dtype: float64 

 tolerances
 [[-0.00425404]
 [ 0.07434088]
 [-0.11931708]
 [ 0.00392613]] 

Pre-Stressed Concrete 

 sample size	 68
win_down     0.233707
marg_cont    0.014940
marg_info    0.029000
win0         0.111886
win_up       0.111886
dtype: float64 

 tolerances
 [[-7.08031074e-02]
 [ 2.12383903e-01]
 [-6.16805236e-02]
 [ 7.81122486e-05]] 

Floods 

 sample size	 84
win_down     0.238591
marg_cont    0.001190
marg_info    0.026025
win0         0.194350
win_up       0.190183
dtype: float64 

 tolerances
 [[-0.03506403]
 [ 0.08183192]
 [-0.09511038]
 [ 0.00187392]] 



In [18]:
solver._moment_matrix

array([[-1,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0],
       [ 0,  0,  1,  0,  0],
       [ 0,  0,  0,  1,  0],
       [ 0,  0,  0,  0, -1]])

In [20]:
.001/np.sqrt(80)

0.00011180339887498948